In [2]:
import requests
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import re

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
#     method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: #bot 접근 못하게 막은데있어어 else뺌
    try:
        resp = requests.request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
            print(limit)
            time.sleep(300) # => random, 1초에 1번
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [4]:
driver.get('https://www.google.com/search?hl=ko&sxsrf=ALeKk02BsJh1j2sMTYpd7ObrTYEPGpcQAA%3A1595765026321&source=hp&ei=InEdX4e0EY_ahwOBqKe4Dw&q=%EC%B1%84%EC%8B%9D+%EC%8B%9D%EB%8B%B9&oq=&gs_lcp=CgZwc3ktYWIQAxgAMgcIIxDqAhAnMgcIIxDqAhAnMgcIIxDqAhAnMgcIIxDqAhAnMgcIIxDqAhAnMgcIIxDqAhAnMgcIIxDqAhAnMgcIIxDqAhAnMgcIIxDqAhAnUABYAGC8FWgBcAB4AIABAIgBAJIBAJgBAKoBB2d3cy13aXqwAQk&sclient=psy-ab')
driver.find_element_by_class_name('i0vbXd').click()

name = []
review = []
first = True

for page in range(2,6):
    print("Page : " + str(page-1))
    dom = BeautifulSoup(driver.page_source, 'lxml')
    for _ in dom.select('div[class=VkpGBb]'):
        value = _.select_one('div[class=dbg0pd]').text.strip() if _.select_one('span[class^=rllt__details]\
        > div').text.strip()[:5]!='리뷰 없음' and _.select_one('span[class^=rllt__details]\
        > div').text.strip().split()[-1] in ['채식', '채식(Vegan)','음식점'] and (_.select_one('span[class^=rllt__details]\
        > div:nth-of-type(2) > span').text.strip()[:5] == '서울특별시' or _.select_one('span[class^=rllt__details]\
        > div:nth-of-type(2) > span:nth-of-type(2)').text.strip()[:5] == '서울특별시') else None
        if value and '뷔페' not in value:
            name.append(value)
            review.append(int(re.sub('[()]','',_.select_one('g-review-stars + span').text.strip())))
    print(len(name))
    print(len(review))
    if page==5:
        break
    driver.find_element_by_xpath('//a[@aria-label="Page ' + str(page) + '"]').click()
    time.sleep(5)
    print()

NameError: name 'driver' is not defined

In [5]:
data = pd.DataFrame()
data['name'] = name
data['review'] = review
data.sort_values(by='review',ascending=False).reset_index(drop=True).iloc[:20]

NameError: name 'name' is not defined

In [ ]:
driver.close()

In [6]:
import pandas as pd

data = pd.DataFrame()
data['name'] = name
data['review'] = review
data.sort_values(by='review',ascending=False).reset_index(drop=True)

NameError: name 'name' is not defined

In [7]:
startrow =1
endrow = 1000
api_key = '9a17a673a90d4ccc9927'
url = 'http://openapi.foodsafetykorea.go.kr/api/' + api_key + '/I2790/json/' + str(startrow) + '/' + str(endrow)
resp = download(url, headers = headers,method= 'GET')
result = resp.json()
total = int(result['I2790']['total_count'])
de = total%1000 #한번에 천개씩 제한 

import sqlite3
con = sqlite3.connect('ex.db')
cur = con.cursor()

cur.executescript('''
    DROP TABLE IF EXISTS factor;
    CREATE TABLE factor(
        NUM           INTEGER NOT NULL,
        DESC_KOR      TEXT NOT NULL ,
        FOOD_GROUP    TEXT ,
        FOOD_CD      TEXT ,
        NUTR_CONT1     FLOAT DEFAULT 0.0,
        NUTR_CONT2     FLOAT DEFAULT 0.0,
        NUTR_CONT3     FLOAT DEFAULT 0.0,
        NUTR_CONT4   FLOAT DEFAULT 0.0,
        NUTR_CONT5   FLOAT DEFAULT 0.0,
        NUTR_CONT6   FLOAT DEFAULT 0.0,
        NUTR_CONT7   FLOAT DEFAULT 0.0,
        NUTR_CONT8   FLOAT DEFAULT 0.0,
        NUTR_CONT9   FLOAT DEFAULT 0.0,
        SERVING_SIZE   INTERGER ,
        MAKER_NAME    TEXT ,
        BGN_YEAR      TEXT ,
        SAMPLING_MONTH_NAME     TEXT ,
        SUB_REF_NAME      TEXT,
        SAMPLING_REGION_NAME   TEXT ,
        GROUP_NAME       TEXT ,
        RESEARCH_YEAR INTERGER DEFAULT 0,
        SAMPLING_REGION_CD  INTERGER DEFAULT 0,
        SAMPLING_MONTH_CD  TEXT ,
        ANIMAL_PLANT    TEXT 
    );
''')

In [8]:
result

{'I2790': {'RESULT': {'MSG': '정상처리되었습니다.', 'CODE': 'INFO-000'},
  'total_count': '29235',
  'row': [{'NUTR_CONT3': '33.5',
    'NUTR_CONT2': '39.7',
    'NUTR_CONT1': '368.8',
    'SERVING_SIZE': '500',
    'FOOD_GROUP': '',
    'MAKER_NAME': '',
    'BGN_YEAR': '',
    'NUTR_CONT9': '0.1',
    'NUTR_CONT8': '1.9',
    'FOOD_CD': 'D00006',
    'NUTR_CONT7': '106.18',
    'NUTR_CONT6': '1264.31',
    'NUTR_CONT5': '16.9',
    'NUTR_CONT4': '8.5',
    'DESC_KOR': '꿩불고기',
    'SAMPLING_MONTH_NAME': '',
    'SUB_REF_NAME': "식약처('16) 제4권",
    'SAMPLING_REGION_NAME': '충주',
    'GROUP_NAME': '구이류',
    'RESEARCH_YEAR': '2019',
    'SAMPLING_REGION_CD': '94',
    'SAMPLING_MONTH_CD': 'D01',
    'NUM': '1',
    'ANIMAL_PLANT': ''},
   {'NUTR_CONT3': '45.9',
    'NUTR_CONT2': '44.9',
    'NUTR_CONT1': '595.61',
    'SERVING_SIZE': '400',
    'FOOD_GROUP': '',
    'MAKER_NAME': '',
    'BGN_YEAR': '',
    'NUTR_CONT9': '0.2',
    'NUTR_CONT8': '6',
    'FOOD_CD': 'D00007',
    'NUTR_CONT7': '193

In [9]:
for _ in result['I2790']['row']:
    cur.execute('''INSERT INTO factor(NUM,DESC_KOR      ,
        FOOD_GROUP ,FOOD_CD ,NUTR_CONT1 ,
        NUTR_CONT2, NUTR_CONT3, NUTR_CONT4, NUTR_CONT5,
        NUTR_CONT6, NUTR_CONT7,  NUTR_CONT8, NUTR_CONT9,
        SERVING_SIZE  ,MAKER_NAME , BGN_YEAR, SAMPLING_MONTH_NAME  ,
        SUB_REF_NAME, SAMPLING_REGION_NAME, GROUP_NAME  ,
        RESEARCH_YEAR, SAMPLING_REGION_CD ,SAMPLING_MONTH_CD, ANIMAL_PLANT) 
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',[
        _['NUM'],_['DESC_KOR'] ,
        _['FOOD_GROUP'] ,_['FOOD_CD'] ,_['NUTR_CONT1'] ,
        _['NUTR_CONT2']  ,_['NUTR_CONT3']  ,_['NUTR_CONT4']  , _['NUTR_CONT5'],
        _['NUTR_CONT6']   ,_['NUTR_CONT7'],  _['NUTR_CONT8'], _['NUTR_CONT9'],
        _['SERVING_SIZE']  ,_['MAKER_NAME'] , _['BGN_YEAR']     ,_['SAMPLING_MONTH_NAME']  ,
        _['SUB_REF_NAME'] , _['SAMPLING_REGION_NAME'] , _['GROUP_NAME']  ,
        _['RESEARCH_YEAR'] ,_['SAMPLING_REGION_CD'] ,_['SAMPLING_MONTH_CD'],_['ANIMAL_PLANT']])

In [12]:
con.commit()

In [14]:
for i in range(int(total/1000)):
    startrow+= 1000
    if i==int(total/1000)-1:
        endrow += (de-1)
    else:
        endrow += 1000
    url = 'http://openapi.foodsafetykorea.go.kr/api/' + api_key + '/I2790/json/' + str(startrow) + '/' + str(endrow)
    resp = download(url, headers = headers,method= 'GET')
    result = resp.json()
    
    for _ in result['I2790']['row']:
        print(_)
    cur.execute('''INSERT INTO factor(NUM,DESC_KOR      ,
        FOOD_GROUP ,FOOD_CD ,NUTR_CONT1 ,
        NUTR_CONT2, NUTR_CONT3, NUTR_CONT4, NUTR_CONT5,
        NUTR_CONT6, NUTR_CONT7,  NUTR_CONT8, NUTR_CONT9,
        SERVING_SIZE  ,MAKER_NAME , BGN_YEAR, SAMPLING_MONTH_NAME  ,
        SUB_REF_NAME, SAMPLING_REGION_NAME, GROUP_NAME  ,
        RESEARCH_YEAR, SAMPLING_REGION_CD ,SAMPLING_MONTH_CD, ANIMAL_PLANT) 
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',[
        _['NUM'],_['DESC_KOR']      ,
        _['FOOD_GROUP'] ,_['FOOD_CD'] ,_['NUTR_CONT1'] ,
        _['NUTR_CONT2']  ,_['NUTR_CONT3']  ,_['NUTR_CONT4']  , _['NUTR_CONT5'],
        _['NUTR_CONT6']   ,_['NUTR_CONT7'],  _['NUTR_CONT8'], _['NUTR_CONT9'],
        _['SERVING_SIZE']  ,_['MAKER_NAME'] , _['BGN_YEAR']     ,_['SAMPLING_MONTH_NAME']  ,
        _['SUB_REF_NAME'] , _['SAMPLING_REGION_NAME'] , _['GROUP_NAME']  ,
        _['RESEARCH_YEAR'] ,_['SAMPLING_REGION_CD'] ,_['SAMPLING_MONTH_CD'],_['ANIMAL_PLANT']])
    time.sleep(1)
    con.commit()
    time.sleep(1)

{'NUTR_CONT3': '17', 'NUTR_CONT2': '', 'NUTR_CONT1': '327', 'SERVING_SIZE': '128', 'FOOD_GROUP': '', 'MAKER_NAME': '뽕뜨락피자', 'BGN_YEAR': '', 'NUTR_CONT9': '', 'NUTR_CONT8': '6', 'FOOD_CD': 'D10729', 'NUTR_CONT7': '', 'NUTR_CONT6': '391', 'NUTR_CONT5': '9', 'NUTR_CONT4': '', 'DESC_KOR': '스윗고구마M', 'SAMPLING_MONTH_NAME': '평균', 'SUB_REF_NAME': "식약처('19)", 'SAMPLING_REGION_NAME': '전국(대표)', 'GROUP_NAME': '피자류', 'RESEARCH_YEAR': '2019', 'SAMPLING_REGION_CD': 'ZZ', 'SAMPLING_MONTH_CD': 'AVG', 'NUM': '1001', 'ANIMAL_PLANT': ''}
{'NUTR_CONT3': '21', 'NUTR_CONT2': '', 'NUTR_CONT1': '438', 'SERVING_SIZE': '163', 'FOOD_GROUP': '', 'MAKER_NAME': '뽕뜨락피자', 'BGN_YEAR': '', 'NUTR_CONT9': '', 'NUTR_CONT8': '9', 'FOOD_CD': 'D10730', 'NUTR_CONT7': '', 'NUTR_CONT6': '628', 'NUTR_CONT5': '9', 'NUTR_CONT4': '', 'DESC_KOR': '페페로니L', 'SAMPLING_MONTH_NAME': '평균', 'SUB_REF_NAME': "식약처('19)", 'SAMPLING_REGION_NAME': '전국(대표)', 'GROUP_NAME': '피자류', 'RESEARCH_YEAR': '2019', 'SAMPLING_REGION_CD': 'ZZ', 'SAMPLING_MONTH_

{'NUTR_CONT3': '7.9', 'NUTR_CONT2': '', 'NUTR_CONT1': '269', 'SERVING_SIZE': '270', 'FOOD_GROUP': '', 'MAKER_NAME': '커피마마', 'BGN_YEAR': '', 'NUTR_CONT9': '', 'NUTR_CONT8': '7', 'FOOD_CD': 'D11759', 'NUTR_CONT7': '', 'NUTR_CONT6': '130.5', 'NUTR_CONT5': '25.4', 'NUTR_CONT4': '', 'DESC_KOR': '홍차라떼 R', 'SAMPLING_MONTH_NAME': '평균', 'SUB_REF_NAME': "식약처('19)", 'SAMPLING_REGION_NAME': '전국(대표)', 'GROUP_NAME': '음료류', 'RESEARCH_YEAR': '2019', 'SAMPLING_REGION_CD': 'ZZ', 'SAMPLING_MONTH_CD': 'AVG', 'NUM': '2001', 'ANIMAL_PLANT': ''}
{'NUTR_CONT3': '10.1', 'NUTR_CONT2': '', 'NUTR_CONT1': '364.5', 'SERVING_SIZE': '345', 'FOOD_GROUP': '', 'MAKER_NAME': '커피마마', 'BGN_YEAR': '', 'NUTR_CONT9': '', 'NUTR_CONT8': '9', 'FOOD_CD': 'D11760', 'NUTR_CONT7': '', 'NUTR_CONT6': '165.7', 'NUTR_CONT5': '35.5', 'NUTR_CONT4': '', 'DESC_KOR': '홍차라떼 L', 'SAMPLING_MONTH_NAME': '평균', 'SUB_REF_NAME': "식약처('19)", 'SAMPLING_REGION_NAME': '전국(대표)', 'GROUP_NAME': '음료류', 'RESEARCH_YEAR': '2019', 'SAMPLING_REGION_CD': 'ZZ', 'S

{'NUTR_CONT3': '13', 'NUTR_CONT2': '', 'NUTR_CONT1': '315', 'SERVING_SIZE': '134', 'FOOD_GROUP': '', 'MAKER_NAME': '파파존스', 'BGN_YEAR': '', 'NUTR_CONT9': '', 'NUTR_CONT8': '4', 'FOOD_CD': 'D12664', 'NUTR_CONT7': '', 'NUTR_CONT6': '745', 'NUTR_CONT5': '7', 'NUTR_CONT4': '', 'DESC_KOR': '하와이안 오리지널®', 'SAMPLING_MONTH_NAME': '평균', 'SUB_REF_NAME': "식약처('19)", 'SAMPLING_REGION_NAME': '전국(대표)', 'GROUP_NAME': '피자류', 'RESEARCH_YEAR': '2019', 'SAMPLING_REGION_CD': 'ZZ', 'SAMPLING_MONTH_CD': 'AVG', 'NUM': '2897', 'ANIMAL_PLANT': ''}
{'NUTR_CONT3': '18', 'NUTR_CONT2': '', 'NUTR_CONT1': '428', 'SERVING_SIZE': '182', 'FOOD_GROUP': '', 'MAKER_NAME': '파파존스', 'BGN_YEAR': '', 'NUTR_CONT9': '', 'NUTR_CONT8': '5', 'FOOD_CD': 'D12665', 'NUTR_CONT7': '', 'NUTR_CONT6': '1011', 'NUTR_CONT5': '10', 'NUTR_CONT4': '', 'DESC_KOR': '하와이안 오리지널(L)', 'SAMPLING_MONTH_NAME': '평균', 'SUB_REF_NAME': "식약처('19)", 'SAMPLING_REGION_NAME': '전국(대표)', 'GROUP_NAME': '피자류', 'RESEARCH_YEAR': '2019', 'SAMPLING_REGION_CD': 'ZZ', 'SAMP

KeyboardInterrupt: 

In [ ]:
for _ in result['I2790']['row']:
    print(_)
    cur.execute('''INSERT INTO factor(NUM,DESC_KOR      ,
        FOOD_GROUP ,FOOD_CD ,NUTR_CONT1 ,
        NUTR_CONT2, NUTR_CONT3, NUTR_CONT4, NUTR_CONT5,
        NUTR_CONT6, NUTR_CONT7,  NUTR_CONT8, NUTR_CONT9,
        SERVING_SIZE  ,MAKER_NAME , BGN_YEAR, SAMPLING_MONTH_NAME  ,
        SUB_REF_NAME, SAMPLING_REGION_NAME, GROUP_NAME  ,
        RESEARCH_YEAR, SAMPLING_REGION_CD ,SAMPLING_MONTH_CD, ANIMAL_PLANT) 
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''',[
        _['NUM'],_['DESC_KOR']      ,
        _['FOOD_GROUP'] ,_['FOOD_CD'] ,_['NUTR_CONT1'] ,
        _['NUTR_CONT2']  ,_['NUTR_CONT3']  ,_['NUTR_CONT4']  , _['NUTR_CONT5'],
        _['NUTR_CONT6']   ,_['NUTR_CONT7'],  _['NUTR_CONT8'], _['NUTR_CONT9'],
        _['SERVING_SIZE']  ,_['MAKER_NAME'] , _['BGN_YEAR']     ,_['SAMPLING_MONTH_NAME']  ,
        _['SUB_REF_NAME'] , _['SAMPLING_REGION_NAME'] , _['GROUP_NAME']  ,
        _['RESEARCH_YEAR'] ,_['SAMPLING_REGION_CD'] ,_['SAMPLING_MONTH_CD'],_['ANIMAL_PLANT']])
con.commit()

In [ ]:
result